## Overview

#### written on 09/30/24

from the existing record, gather more records.
randomly choose accounts from accounts table.
fetch their recent game records.
if the record has not been added yet, add it.

## Import

In [1]:
import time
import importlib
import api_caller
import db_connector

## API call setting

In [2]:
# Reload the module to ensure the latest changes are reflected
importlib.reload(api_caller)

caller = api_caller.APICaller()

# Test your function call
# result = caller.get_account_from_riot_id("NoMatterWhoUR", "NA1")
# print(result)

## DB connection setting

In [3]:
# Reload the module to ensure the latest changes are reflected
importlib.reload(db_connector)

connector = db_connector.DBConnector()

## Query preparation

In [4]:
# matches
# conflict won't happen because it will be filtered before calling an api.

'''
original query:
INSERT INTO matches (match_id, game_duration, remake, game_start, game_end, version_1, version_2)
VALUES (%s, %s, %s, %s, %s, %s, %s)
ON CONFLICT (match_id)
DO NOTHING
'''

matches_query = {
    "INSERT": "INTO matches",
    "ON CONFLICT": "(match_id)",
    "DO NOTHING": ""
}

matches_col = ["match_id", "game_duration", "remake", "game_start", "game_end", "version_1", "version_2"]

In [5]:
# accounts
# conflict may take place and it should be updated (name or tag change, or level up)
# but puuid won't change.

'''
original query:
INSERT INTO accounts (puuid, user_name, tag, lvl)
VALUES (%s, %s, %s, %s)
ON CONFLICT (puuid)
DO UPDATE SET user_name = %s, tag = %s, lvl = %s
'''

accounts_query = {
    "INSERT": "INTO accounts",
    "ON CONFLICT": "(puuid)",
    "DO UPDATE SET": ""
}

accounts_col = ["puuid", "user_name", "tag", "lvl"]
accounts_update_col = ["user_name", "tag", "lvl"]

In [6]:
# match_account
# conflict won't happen, because match will be filtered.

'''
original query:
INSERT INTO match_account (match_id, puuid, win, champ_name, champ_id, champ_lvl, gold, cs, kill, death, assist, kda,
damag_to_total, damaged_by, heal_team, shield_team, time_cc_to, time_ccd_by, kill_participation_pct, damage_pct,
damaged_pct, spell_1, spell_2, rune_main, rune_sub, rune_main_1, rune_main_2, rune_main_3, rune_main_4, rune_sub_1,
rune_sub_2, stat_off, stat_flex, stat_def, item_1, item_2, item_3, item_4, item_5, item_6, double_kill, triple_kill,
quadra_kill, penta_kill, damage_to_physical, damage_to_magic, damage_to_true, damaged_mitigated, damaged_self_healed)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) 
ON CONFLICT (match_id, puuid)
DO NOTHING
'''

match_account_query = {
    "INSERT": "INTO match_account",
    "ON CONFLICT": "(match_id, puuid)",
    "DO NOTHING": ""
}

# please work...
match_account_col_string = '''match_id, puuid, win, champ_name, champ_id, champ_lvl, gold, cs, kill, death, assist, kda,
damage_to_total, damaged_by, heal_team, shield_team, time_cc_to, time_ccd_by, kill_participation_pct, damage_pct,
damaged_pct, spell_1, spell_2, rune_main, rune_sub, rune_main_1, rune_main_2, rune_main_3, rune_main_4, rune_sub_1,
rune_sub_2, stat_off, stat_flex, stat_def, item_1, item_2, item_3, item_4, item_5, item_6, double_kill, triple_kill,
quadra_kill, penta_kill, damage_to_physical, damage_to_magic, damage_to_true, damaged_mitigated, damaged_self_healed'''
match_account_col = match_account_col_string.replace("\n", " ").split(", ")
# print(match_account_col)

## Main code

In [7]:
# number of accounts to fetch info of.
num_puuids = 

accounts_select_query = {
    "SELECT": "puuid",
    "FROM": "accounts",
    "ORDER BY": "random()",
    "LIMIT": str(num_puuids)
}
puuids = connector.select_query(accounts_select_query)

In [8]:
# fetch existing matches. make sure collected matches are new, because each match will take 1 api call.
# turn them into dict to save time (exchange time and memory): time is #1 concern.

match_ids_select_query = {
    "SELECT": "match_id",
    "FROM": "matches",
}
match_ids = connector.select_query(match_ids_select_query)
match_id_lookup = set()
for match_id in match_ids:
    match_id_lookup.add(match_id[0])
# print(len(match_id_lookup))

In [9]:
# fetch match records for selected puuids.
match_list = []
for idx, puuid in enumerate(puuids):

    end_time = 0
    new_match_list = caller.get_match_id_list_from_puuid(puuid[0], end_time = end_time, count = 100)
                                                         
    while (len(new_match_list) == 100):
        match_list += new_match_list
        print(F"{len(new_match_list)} matches have been added.")
        # 1 second before the oldest record's ending time 
        end_time = caller.get_match_from_match_id(new_match_list[-1])["info"]['gameEndTimestamp'] // 1000 - 1
        new_match_list = caller.get_match_id_list_from_puuid(puuid[0], end_time = end_time, count = 100)
                                                             
    # insert the last list that is less than 100 records
    match_list += new_match_list
    print(F"{len(new_match_list)} matches have been added.")
                    
    print(F"Found all match records for {idx + 1} puuids. Current amount of the collected records: {len(match_list)}")
print("all match records for given puuids have been fully fetched.")

100 matches have been added.
3 matches have been added.
Found all match records for 1 puuids. Current amount of the collected records: 103
100 matches have been added.
100 matches have been added.
100 matches have been added.
100 matches have been added.
49 matches have been added.
Found all match records for 2 puuids. Current amount of the collected records: 552
80 matches have been added.
Found all match records for 3 puuids. Current amount of the collected records: 632
all match records for given puuids have been fully fetched.


In [10]:
# fetch data for each match.
skip_cnt = 0
old_cnt = 0
for idx, match in enumerate(match_list):

    # skip if dup
    if match in match_id_lookup:
        skip_cnt += 1

        # but still show the stat.
        if (idx + 1) % 10 == 0:
            print(F"{idx + 1} matches have been processed:") 
            print(F"  {skip_cnt} matches have been dup and skipped.")
            print(F"  {old_cnt} matches have been before version 14.1, added to match but not to match_account.")
            print(F"  {idx + 1 - skip_cnt - old_cnt} new matches have been added.") 
            pct = "{:.2f}".format((idx + 1 - skip_cnt - old_cnt) / (idx + 1) * 100)
            print(F"  The chance of being new record has been {pct}%.")
        continue
    
    # new record
    data = caller.get_match_from_match_id(match)

    # version number splitted by .
    version = data["info"]["gameVersion"].split(".")

    # for old data, just skip it over.
    # once enough data is stored in match, determine the starting point of 14.1 patch to check before calling api.
    # skip adding account as well because it may require extra api calls for those before 13.23.
    if int(version[0]) < 14:
        old_cnt += 1
        
        # but still show the stat.
        if (idx + 1) % 10 == 0:
            print(F"{idx + 1} matches have been processed:") 
            print(F"  {skip_cnt} matches have been dup and skipped.")
            print(F"  {old_cnt} matches have been before version 14.1, added to match but not to match_account.")
            print(F"  {idx + 1 - skip_cnt - old_cnt} new matches have been added.") 
            pct = "{:.2f}".format((idx + 1 - skip_cnt - old_cnt) / (idx + 1) * 100)
            print(F"  The chance of being new record has been {pct}%.")

        continue

    # conversion to epoch second
    start_epoch = data["info"]["gameStartTimestamp"] // 1000
    end_epoch = data["info"]["gameEndTimestamp"] // 1000

    # insert matches table
    matches_val = (
        data["metadata"]["matchId"],
        data["info"]["gameDuration"],
        str(data["info"]["participants"][0]["gameEndedInEarlySurrender"]),
        time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(start_epoch)),
        time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(end_epoch)),
        version[0],
        version[1]
    )

    connector.insert_query(matches_query, matches_col, matches_val)

    
    # "killParticipation" field doesn't exist for the team with 0 kill
    no_kill_participation_team1 = False
    no_kill_participation_team2 = False

    if data["info"]["participants"][0]["kills"] == 0 and data["info"]["participants"][1]["kills"] == 0 and data["info"]["participants"][2]["kills"] == 0 and data["info"]["participants"][3]["kills"] == 0 and data["info"]["participants"][4]["kills"] == 0:
        no_kill_participation_team1 = True
    if data["info"]["participants"][5]["kills"] == 0 and data["info"]["participants"][6]["kills"] == 0 and data["info"]["participants"][7]["kills"] == 0 and data["info"]["participants"][8]["kills"] == 0 and data["info"]["participants"][9]["kills"] == 0:
        no_kill_participation_team2 = True
        
    for i in range(10):

    
        accounts_val = (data["info"]["participants"][i]["puuid"], 
                        data["info"]["participants"][i]["riotIdGameName"],
                        data["info"]["participants"][i]["riotIdTagline"],
                        data["info"]["participants"][i]["summonerLevel"],
                        data["info"]["participants"][i]["riotIdGameName"],
                        data["info"]["participants"][i]["riotIdTagline"],
                        data["info"]["participants"][i]["summonerLevel"]
        )

        # if no kill participation is marked, manually set it to 0.
        if i < 5 and no_kill_participation_team1 == True:
            kill_participation = 0
        elif i >= 5 and no_kill_participation_team2 == True:
            kill_participation = 0
        else:
            kill_participation = data["info"]["participants"][i]["challenges"]["killParticipation"]
        
        match_account_val = (data["metadata"]["matchId"],
                             data["info"]["participants"][i]["puuid"],
                             str(data["info"]["participants"][i]["win"]),
                             data["info"]["participants"][i]["championName"],
                             data["info"]["participants"][i]["championId"],
                             data["info"]["participants"][i]["champLevel"],
                             data["info"]["participants"][i]["goldEarned"],
                             data["info"]["participants"][i]["totalMinionsKilled"],
                             data["info"]["participants"][i]["kills"],
                             data["info"]["participants"][i]["deaths"],
                             data["info"]["participants"][i]["assists"], 
                             data["info"]["participants"][i]["challenges"]["kda"],
                             data["info"]["participants"][i]["totalDamageDealtToChampions"],
                             data["info"]["participants"][i]["totalDamageTaken"],
                             data["info"]["participants"][i]["totalHealsOnTeammates"],
                             data["info"]["participants"][i]["totalDamageShieldedOnTeammates"],
                             data["info"]["participants"][i]["timeCCingOthers"],
                             data["info"]["participants"][i]["totalTimeCCDealt"],
                             kill_participation,
                             data["info"]["participants"][i]["challenges"]["teamDamagePercentage"],
                             data["info"]["participants"][i]["challenges"]["damageTakenOnTeamPercentage"],
                             data["info"]["participants"][i]["summoner1Id"],
                             data["info"]["participants"][i]["summoner2Id"],
                             data["info"]["participants"][i]["perks"]["styles"][0]["style"],
                             data["info"]["participants"][i]["perks"]["styles"][1]["style"],
                             data["info"]["participants"][i]["perks"]["styles"][0]["selections"][0]["perk"],
                             data["info"]["participants"][i]["perks"]["styles"][0]["selections"][1]["perk"],
                             data["info"]["participants"][i]["perks"]["styles"][0]["selections"][2]["perk"],
                             data["info"]["participants"][i]["perks"]["styles"][0]["selections"][3]["perk"],
                             data["info"]["participants"][i]["perks"]["styles"][1]["selections"][0]["perk"],
                             data["info"]["participants"][i]["perks"]["styles"][1]["selections"][1]["perk"],
                             data["info"]["participants"][i]["perks"]["statPerks"]["offense"],
                             data["info"]["participants"][i]["perks"]["statPerks"]["flex"],
                             data["info"]["participants"][i]["perks"]["statPerks"]["defense"],
                             data["info"]["participants"][i]["item1"],
                             data["info"]["participants"][i]["item2"],
                             data["info"]["participants"][i]["item3"],
                             data["info"]["participants"][i]["item4"],
                             data["info"]["participants"][i]["item5"],
                             data["info"]["participants"][i]["item6"],
                             data["info"]["participants"][i]["doubleKills"],
                             data["info"]["participants"][i]["tripleKills"],
                             data["info"]["participants"][i]["quadraKills"],
                             data["info"]["participants"][i]["pentaKills"],
                             data["info"]["participants"][i]["physicalDamageDealtToChampions"],
                             data["info"]["participants"][i]["magicDamageDealtToChampions"],
                             data["info"]["participants"][i]["trueDamageDealtToChampions"],
                             data["info"]["participants"][i]["damageSelfMitigated"],
                             # self heal = total heal - team heal
                             data["info"]["participants"][i]["totalHeal"] - data["info"]["participants"][i]["totalHealsOnTeammates"]
        )
        connector.insert_query(accounts_query, accounts_col, accounts_val, accounts_update_col)
        connector.insert_query(match_account_query, match_account_col, match_account_val)
    
    if (idx + 1) % 10 == 0:
        print(F"{idx + 1} matches have been processed:") 
        print(F"  {skip_cnt} matches have been dup and skipped.")
        print(F"  {old_cnt} matches have been before version 14.1, added to match but not to match_account.")
        print(F"  {idx + 1 - skip_cnt - old_cnt} new matches have been added.") 
        pct = "{:.2f}".format((idx + 1 - skip_cnt - old_cnt) / (idx + 1) * 100)
        print(F"  The chance of being new record has been {pct}%.")

    connector.commit()

10 matches have been processed:
  0 matches have been dup and skipped.
  10 matches have been before version 14.1, added to match but not to match_account.
  0 new matches have been added.
  The chance of being new record has been 0.00%.
20 matches have been processed:
  0 matches have been dup and skipped.
  20 matches have been before version 14.1, added to match but not to match_account.
  0 new matches have been added.
  The chance of being new record has been 0.00%.
30 matches have been processed:
  0 matches have been dup and skipped.
  30 matches have been before version 14.1, added to match but not to match_account.
  0 new matches have been added.
  The chance of being new record has been 0.00%.
40 matches have been processed:
  0 matches have been dup and skipped.
  40 matches have been before version 14.1, added to match but not to match_account.
  0 new matches have been added.
  The chance of being new record has been 0.00%.
50 matches have been processed:
  0 matches have

In [11]:
connector.commit()
connector.end()
print('all done')